<img src="http://www.exalumnos.usm.cl/wp-content/uploads/2015/06/Isotipo-Negro.gif" title="Logo USM" width="20%" height="20%" />


<hr style="height:2px;border:none"/>
<h1 align='center'> INF-393 Máquinas de Aprendizaje II-2018 </h1>

<H3 align='center'> Tarea 1  </H3>
<hr style="height:2px;border:none"/>
<p style="text-align:center"> Gabriel Ortega 201573580-8 </p>
<p style="text-align:center"> Benjamín Riquelme 201573507-7 </p>

<a id="primero"></a>
## 1. Aprendizaje con regresión lineal.

El modelo de regresión lineal  es una combinación lineal entre variables independientes para obtener otra variable, dependiente de éstas. Lo cual puede resultar bastante simple, pero, hoy en día, ha podido ser aplicado a varios problemas con buenos resultados, como predicción en finanzas y en medicina. Sin embargo, también puede ser un medio para aplicar un modelo más grande, por ejemplo utilizarlo para que, con el resuido, detectar *outliers*, rellenar vacíos/datos incompletos o aprender un *score* para ranquear objetos, lo que haremos en esta sección.

<img src="http://chanakya.ca/wp-content/uploads/2018/05/EstimateMultipleLinearRegressionCoefficientsExample_01.png" title="Grafico 1" height="15%" />


El problema de *learning to rank* es aplicado comúnmente en *Information Retrieval* (IR). Sin embargo, el aprender ésta función puede ser crucial para modelar la importancia de distintos objetos.  


En esta actividad trabajaremos con el problema de predecir el *ranking* mundial de una Universidad en base a distintas características de ésta (dataset *World University Rankings*, a través del siguiente __[link](https://www.kaggle.com/mylesoneill/world-university-rankings)__) en la plataforma de *Kaggle*. En este problema el *ranking* es una medición de qué tan buena es la universidad e intentaremos predecirla a través un modelo simple de regresión lineal. En particular, dentro de los miles de diferentes sistemas de rankings, nacionales e internacionales, entre los cuales comúnmente existen desacuerdos entre ellos, trabajaremos con el ranking ampliamente considerado como uno de las más influyentes y ampliamente observadas: *Times Higher Education World University* .


> a) Cargue los datos a analizar, descargándolos desde la plataforma como se indicó, en formato *dataframe pandas*. Descríbalos adecuadamente, ya sea la variable dependiente o las independientes, si es que lo son.

In [ ]:
import pandas as pd
df = pd.read_csv("timesData.csv")
print("Dimensiones originales: {0}".format(df.shape))
df.head()

En el dataset podemos distinguir 2 tipos de columnas, un tipo son los datos de la universidad (a los que referiremos como *descriptores*) como el nombre, país en el que se encuentra, cantidad de estudiantes,etc. Y el otro tipo corresponde a los puntajes del ranking, esta columnas son:

<ul>
<li>teaching
<li>international
<li>research
<li>citations
<li>income
<li>total_score    
</ul>

La última columna mencionada corresponde a la variable dependiente, el "Y" que queremos predecir, además pareciera que las universidades están ordenadas por este puntaje, para comprobarlo se hará una función que verifique que cada elemento de la lista sea mayor o igual al siguiente. 

In [ ]:
rankings = df["total_score"]
def discrepancias(lista):
    largo = len(lista)
    for i in range(largo-1):
        if lista[i] < lista[i+1]:
            print(i) #Aqui se muestra donde ocurre la discrepancia
    return 
discrepancias(rankings)


Al ejecutar la función se pueden notar unas discrepancias en 5 índices del dataset y esto es debido a la separación por los años que se encuentra en éste. A continuación, se verá en mayor detalle lo que sucede alrededor del índice indicado por la función anterior para verificar lo dicho con anterioridad

In [ ]:
df[["university_name","total_score","year"]][197:202]

In [ ]:
df[["university_name","total_score","year"]][599:604]

In [ ]:
df[["university_name","total_score","year"]][999:1004]

In [ ]:
df[["university_name","total_score","year"]][1399:1404]

In [ ]:
df[["university_name","total_score","year"]][1800:1805]

Como se puede observar en los distintos subconjuntos presentados, las discrepancias son porque los años varían en el dataset, entonces lo que se tiene son los resultados de 5 años distintos en las universidades, como en el resto de las filas no se presentan discrepancias entonces si se puede decir que los puntajes de la columna "total_score" ordenan a las universidades según el año en que se realizó la evaluación. Podemos identificar la variable "total_score" como la variable dependiente y todas las demas como variables independientes, obviando el atributo "world_rank" ya que este puede considerarse como un índice si es que ordenamos las universidades por su puntaje total.

In [ ]:
sorted(df["total_score"].unique())

Como se puede ver de la celda de código anterior no hay valores NA en la columna que nos interesa predecir, solamente valores '-' que representan el puntaje faltante y el grupo que se separará del resto para predecir estos puntajes no calculados.

> b) Debido a la estructura será necesario realizar un leve pre-proceso. Existen vacíos entre los datos o valores '-', por lo que será necesario eliminarlos (*o si piensa una mejor manera de manejar ésto puede hacerlo, se verá reflejado en su nota*). Además de ésto deje los datos con *score unkown* o '-' en un conjunto *target* separado, *unlabeled data* (éste será el objetivo del entrenamiento) ¿Cuántos datos quedan en cada conjunto? 

In [ ]:
def convertToInt(x):
    try:
        x = int(x)
    except:
        x = 0
    return x
df.dropna(axis=0,inplace=True,how='any') #borra nan
df["total_score"] = df["total_score"].apply(lambda x: x.replace('-','unknown')) #rellena 
df = df[~(df == '-').any(axis=1)] #elimina filas con valores nulos

nuevo_df  = pd.get_dummies(df, columns=["country"]) #crea nuevas columnas de booleanos indicando si es del pais o no

nuevo_df['female'] = nuevo_df['female_male_ratio'].str.split(':', expand=True)[0].apply(convertToInt)
nuevo_df['male'] = nuevo_df['female_male_ratio'].str.split(':', expand=True)[1].apply(convertToInt) #separa hombres y mujeres
nuevo_df['female_male_ratio'] =  np.where(nuevo_df['male'] == 0, 0, nuevo_df['female']/nuevo_df['male']) #si no hay (rellena 0) 
nuevo_df['num_students'] = nuevo_df['num_students'].apply(lambda x: int(str(x).replace(',','')))
nuevo_df['international_students'] = nuevo_df['international_students'].apply(lambda x: int(str(x).replace('%','')))

df_test = nuevo_df[nuevo_df["total_score"]=='unknown']  #para predecir al final
nuevo_df =  nuevo_df[nuevo_df["total_score"]!='unknown'] #elimina unknown rank..
print("Dimensiones del dataset de prueba: {0}".format(df_test.shape))
print("Dimensiones del dataset de entrenamiento: {0} ".format(nuevo_df.shape))
nuevo_df.head()

Considerando todos los atributos como importantes, luego del filtrado se tienen 2174 datos que corresponden a un 75% de los datos originalmente recopilados. Se pudieron evitar algunas pérdidas si en las columnas de puntaje se hubiera reemplazado un dato faltante por la media correspondiente a ese año, pero esto es algo arriesgado de hacer porque reemplazar estos valores que consideramos importantes afectaría el ranking otorgado por el learner al final para bien o para mal.

Al final tenemos un dataset de prueba con 1220 elementos y un dataset de entrenamiento con 954 datos.

> c) Cree las matrices de cada conjunto con las que trabajará. Además de ésto separe el conjunto de pruebas fijo que se utilizará, recuerde que éste no puede ser utilizado. Si estima conveniente también cree conjunto de validación.

In [ ]:
Y = nuevo_df['total_score'].values
X = nuevo_df.drop(["total_score","world_rank","university_name"],axis=1).values
X_test = df_test.drop(["total_score","world_rank","university_name"],axis=1).values
Y = Y.astype('float32')
X = X.astype('float32')
X_test = X_test.astype('float32')
X.shape

from sklearn.model_selection import train_test_split
#Esto hace que se generen conjuntos de prueba y testeo aleatorios utilizando los algoritmos de sklearn
#random state es como la semilla para replicar los conjuntos. 
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

> d) Normalice los datos antes de trabajar. Explique la importancia/conveniencia de realizar ésto.

La importancia de normalizar los datos es que esto permite ajustar los rangos de los valores que poseen estos datos y dejarlos similares. Considerando que se tendrán constantes acompañando a las etiquetas es de esperar que etiquetas de órdenes de magnitud grande tengan constantes de ordenes de magnitud grandes y viceversa opacando variables cuyos ordenes de magnitud difieren mucho entre sí. Con la normalización se pueden dejar todas las variables involucradas en los mismo órdenes de magnitud (una normal con media 0 y desviación standar 1 en el caso del standarscaler de sklearn)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(with_mean=True, with_std=True)
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

> e) Realice una regresión lineal de mı́nimos cuadrados básica. Mida el residuo de cada predicción en cada dato y haga un gráfico de éste ¿Qué indica lo observado?

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LinearRegression as LR
linreg = LR(fit_intercept=True, n_jobs=1)
linreg.fit(X_train_scaled,y_train)
import seaborn as sns
%matplotlib inline
res = y_train-linreg.predict(X_train_scaled)
sns.distplot(res)

In [ ]:
a = linreg.predict(X_train_scaled)


> f) Construya una tabla con los pesos, Z-score y F-score correspondientes a cada predictor (variable), compare estos valores. ¿Qué sucede si hacemos un raking de los atributos en base al peso obtenido en la regresión? Compare y comente ¿Qué variables están más correlacionadas con la respuesta? Si usáramos un nivel de significación del 5%. ¿Qué es lo que observa y cuál puede ser la causa?

## 2. Análisis de audios como datos brutos

Distintos tipos de datos han sido tratados en el área de Machine Learning, donde el análisis de estos y
el manejo para poder dejarlos en una representación que se pueda entregar como entrada al algoritmo es
crucial. El manejo sobre los datos brutos se denomina pre-procesamiento y existen distintos dependiento del
tipo de datos y los distintos dominios de problemas, tales como imágenes, audios, texto.  
En esta actividad se trabajará con datos de audios los cuales son directamente extraı́dos desde datos fuentes
*.wav*, lo que corresponde a una señal de sonido en diferentes tiempos.

<img src="https://cdn.shopify.com/s/files/1/0977/4240/products/il_fullxfull.1054777221_nym4.jpg?v=1527718941" width="40%" />



El *dataset* se denomina **Heartbeat Sounds**[[3]](#refs) y es presentado en la plataforma Kaggle a través del siguiente  __[link](https://www.kaggle.com/kinguistics/heartbeat-sounds)__. Este dataset consta de grabaciones de sonidos de latidos cardı́acos normales y anormales, con distintas categorı́as para los latidos anormales. Para la tarea se trabajará con el *dataset A* presente en la data, el cual corresponde a datos generados desde la vı́a pública mediante la aplicación de Iphone iStethoscope Pro. El objetivo será el de clasificar cada sonido como latido cardı́aco normal o una de las las subcategorı́as de anormal (*Murmur, Extra Heart Sound, Artifact*), por lo que se trata de un problema de clasificación múltiple con 4 clases. Las distintas clasificaciones para los sonidos son explicadas en el sitio de Kaggle.

In [22]:
#Para trabajar con los archivos WAV
i = 0
from scipy.io import wavfile
def clean_filename(fname, string):
    file_name = fname.split('/')[1]
    if file_name[:2] == '__':
        file_name = string + file_name
    return file_name
SAMPLE_RATE = 44100
def load_wav_file(name, path):
    s, b = wavfile.read(path + name)
    assert s == SAMPLE_RATE
    return b

> a) Construya un dataframe con los datos a analizar. Describa el dataset y determine cuántos registros hay
por clase.

In [2]:
import pandas as pd
import numpy as np
heartbeats_df = pd.read_csv('set_a.csv')
print("Tipos de latidos: {0}".format(heartbeats_df["label"].unique()))
heartbeats_df.head()

Tipos de latidos: ['artifact' 'extrahls' 'murmur' 'normal' nan]


,dataset,fname,label,sublabel
0,a,set_a/artifact__201012172012.wav,artifact,NaN
1,a,set_a/artifact__201105040918.wav,artifact,NaN
2,a,set_a/artifact__201105041959.wav,artifact,NaN
3,a,set_a/artifact__201105051017.wav,artifact,NaN
4,a,set_a/artifact__201105060108.wav,artifact,NaN


En el dataset se pueden distinguir cuatro columnas, de las cuales dos son de interés, el nombre del archivo contenido en la columna *fname* y el *label* que corresponde al tipo del latido. La columna *sublabel* no es de interés puesto que es una columna para utilizar en el segundo set de datos "set_b" con el cual no se trabajará en esta sección, era un indicador de ruido de fondo en el .wav del latido; la columna dataset tampoco es útil porque solamente indica en qué dataset estamso trabajando y todos los registros de esta columna son "a" lo que no aporta ninguna utilidad.

In [3]:
heartbeats_df = heartbeats_df[["fname","label"]]
heartbeats_df.head()

,fname,label
0,set_a/artifact__201012172012.wav,artifact
1,set_a/artifact__201105040918.wav,artifact
2,set_a/artifact__201105041959.wav,artifact
3,set_a/artifact__201105051017.wav,artifact
4,set_a/artifact__201105060108.wav,artifact


In [4]:
print("Valores por cada etiqueta:")
heartbeats_df.label.value_counts(dropna=False)

Valores por cada etiqueta:


NaN         52
artifact    40
murmur      34
normal      31
extrahls    19
Name: label, dtype: int64

In [5]:
pd.isnull(heartbeats_df["fname"]).unique()

array([False])

En el conjunto de datos se tienen 52 etiquetas con valores NA, sin embargo, se cuenta con todos los nombres de los archivos .wav por lo que es posible utilizar el conjunto de etiquetas NA como un conjunto para predecir las etiquetas.

> b) Lea los archivos *.wav* y transformelos en secuencias de tiempo. Realice un *padding* de ceros al final de cada secuencia para que todas queden representadas con la misma cantidad de elementos, explique la importancia de realizar este paso.

In [31]:
def padd_zeros(array,length):
    aux = np.zeros(length)
    aux[:array.shape[0]] = array
    return aux
#cambia los nombres de archivos con etiqueta NA a unlabelled
new_heartbeatdf =pd.DataFrame({'file_name' : heartbeats_df['fname'].apply(clean_filename,string='Aunlabelledtest')})

#crea la serie de tiempo
new_heartbeatdf['time_series'] = new_heartbeatdf['file_name'].apply(load_wav_file, path="./set_a/")

#crea la columna del largo de las series
new_heartbeatdf['len_series'] = new_heartbeatdf['time_series'].apply(len)

#Hace el padding en las series de tiempo para que todas queden con el mismo largo
new_heartbeatdf['time_series']=new_heartbeatdf['time_series'].apply(padd_zeros,length=max(new_heartbeatdf['len_series']))
new_heartbeatdf.head()

,file_name,time_series,len_series
0,artifact__201012172012.wav,"[1.0, -3.0, -1.0, -7.0, -9.0, -2.0, -6.0, -5.0...",396900
1,artifact__201105040918.wav,"[-2.0, 3.0, -4.0, 4.0, -3.0, 2.0, -1.0, 0.0, 0...",396900
2,artifact__201105041959.wav,"[6.0, -4.0, -9.0, -1.0, -4.0, 1.0, -5.0, 2.0, ...",396900
3,artifact__201105051017.wav,"[-85.0, -198.0, -214.0, -173.0, -177.0, -206.0...",396900
4,artifact__201105060108.wav,"[53.0, -35.0, 47.0, 170.0, 340.0, 436.0, 535.0...",396900


La gracia de hacer padding de ceros es que al tener series temporales, no todas serán de la misma longitud lo que puede afectar al momento de trabajar con estas, entonces a las series de menor largo se les agregan ceros al final hasta que sean del mismo largo que la más larga para tener todos los registros con el mismo largo, el cero es para indicar que no hay atributo *i* en ese instante 

## 3. Análisis de emociones en *tweets*

El análisis de emociones o sentimientos se refiere al proceso de extraer información acerca de la actitud que una persona (o grupo de ellas) manifiesta, en un determinado medio o formato digital, con respecto a un tópico o contexto de comunicación. Uno de los casos más estudiados corresponde a determinar la polaridad de un trozo de texto, es decir, clasificar una determinada evaluación escrita (review ), en que una persona manifiesta una opinión, como positiva, negativa o neutral. Esto también ha sido extendido a otros medios, como lo es analizar la polaridad de textos en redes sociales.

<img src="https://image.flaticon.com/sprites/new_packs/132222-color-emotions-assets.png" width="40%" />



Para esta actividad se trabajará con un datasets de tweets ofrecidos por CrowdFlower[[8]](#refs). Cada *tweet* está
asociado a una emoción en particular, donde el conjunto de emociones se trabajarán como mutuamente excluyentes, siendo un problema de múltiples clases.

Los datos pueden ser descargados ejecutando el siguiente código en sistema Unix:
```
wget https://www.crowdflower.com/wp-content/uploads/2016/07/text_emotion.csv
```
